# Ceneo Scraper
### Import bibliotek

In [11]:
import requests
from bs4 import BeautifulSoup

In [5]:
product_id = "151210226"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

### Wysłanie do serwera ceneo.pl żądania dostępu do strony z opiniami o produkcie

In [7]:
response = requests.get(url)
response.status_code

200

In [22]:
content = response.text
page = BeautifulSoup(content, "html.parser")
opinions = page.select("div.js_product-review")
for opinion in opinions:
    print(opinion)

<div class="user-post user-post__card js_product-review" data-entry-id="17886609">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/2.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
w...7</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2023-09-10 11:53:31">8 miesięcy temu, </time>
<time datetime="2023-09-01 06:41:04">po 2 tygodniach</time> użytkowania            </span>
</span>
<div class="user-post__text">Bardzo dobry sprzed poleca